# Getting Started with PnP Core SDK - Interactive login

This solution demonstrates how the PnP Core SDK can be used in a Polyglot notebook. In this sample we're showing you how you can use PnP Core SDK using the minimal amount of configuration and code.

This notebook is a demonstration based from the minimal console sample to running fragments of code in a Polyglot notebook, that contains WORKING code and returns results base don a secure connection to your tenant

Reference Sample: https://pnp.github.io/pnpcore/demos/Demo.Console.Minimal/readme.html

## Prerequisites

### Getting started

Before this Polyglot notebook can run you need to:
- Install the latest [Visual Studio Code](https://code.visualstudio.com/).
- Install the latest [.NET 6 SDK](https://dotnet.microsoft.com/download/dotnet/6.0).
- Install the Polyglot Notebooks extension from the [marketplace](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode).

### Setup an Azure AD app for accessing SharePoint

The one thing to configure before you can use this sample is an Azure AD application:

1. Navigate to https://aad.portal.azure.com
2. Click on **Azure Active Directory**, followed by navigating to **App registrations**
3. Add a new application via the **New registration** link
4. Give your application a name, e.g. PnPCoreSDKConsoleDemo and add **http://localhost** as redirect URI. Clicking on **Register** will create the application and open it
5. Take note of the **Application (client) ID** value, you'll need it in the next step
6. Click on **API permissions** and add these **delegated** permissions
   1. Microsoft Graph -> Sites.Manage.All
   2. SharePoint -> AllSites.Manage
7. Consent the application permissions by clicking on **Grant admin consent**


### Setting up a connection in the notebook and reference the libraries

Firstly, for the notebook to run, you need to get libraries from nuget and setup the using statements with the following call:

In [1]:
// Only install PnP.Core.Auth, this will pull in PnP.Core
#r "nuget: PnP.Core.Auth,*-*"
#r "nuget: Microsoft.Extensions.Hosting, 6.0.0"

using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Hosting;
using PnP.Core.Auth;
using PnP.Core.Services;

Installed Packages Microsoft.Extensions.Hosting, 6.0.0 PnP.Core.Auth, 1.8.10-nightly

### Set the connection variables

Replace each of these following variables with values from your Azure AD application and you're own SharePoint site. 

In [2]:
// Replace the client ID with the Application ID from the Azure AD app registration
string clientId = "c545f9ce-1c11-440b-812b-0b35217d9e83";

// Set the URL to the SharePoint site you want to connect to
string siteUrl = "https://bertonline.sharepoint.com/sites/prov-1";

Configure the host using the default authentication provider, there are other providers, to see more about the alternatives, check out the [PnP Core SDK documentation](https://pnp.github.io/pnpcore/using-the-sdk/configuring%20authentication.html). 

In [3]:
// Creates and configures the host
var host = Host.CreateDefaultBuilder()
    .ConfigureServices((context, services) => 
    {
        // Add PnP Core SDK
        services.AddPnPCore(options =>
        {
            // Configure the interactive authentication provider as default
            options.DefaultAuthenticationProvider = new InteractiveAuthenticationProvider()
            {
                ClientId = clientId,
                RedirectUri = new Uri("http://localhost")
            };
        });
    })
    .UseConsoleLifetime()
    .Build();

// Start the host
await host.StartAsync();

info: Microsoft.Hosting.Lifetime[0]
      Application started. Press Ctrl+C to shut down.
info: Microsoft.Hosting.Lifetime[0]
      Hosting environment: Production
info: Microsoft.Hosting.Lifetime[0]
      Content root path: c:\Users\bjansen\Downloads


# The application

Now that we have setup the prerequisites, we can start using the PnP Core SDK in the notebook.

In [4]:
using (var scope = host.Services.CreateScope())
{
    // Ask an IPnPContextFactory from the host
    var pnpContextFactory = scope.ServiceProvider.GetRequiredService<IPnPContextFactory>();

    // Create a PnPContext
    using (var context = await pnpContextFactory.CreateAsync(new Uri(siteUrl)))
    {
        // Load the Title property of the site's root web
        await context.Web.LoadAsync(p => p.Title);
        Console.WriteLine($"The title of the web is {context.Web.Title}");
    }
}

The title of the web is prov-1
